In [1]:
import pandas as pd

file_path = r"C:\Users\HR\Desktop\Workspace\데이터톤\마케팅\merged_data.csv"
df = pd.read_csv(file_path)

print(df.head())

                  event_time event_type  product_id          category_id  \
0  2019-10-01 00:00:00+00:00       cart     5773203  1487580005134238553   
1  2019-10-01 00:00:03+00:00       cart     5773353  1487580005134238553   
2  2019-10-01 00:00:07+00:00       cart     5881589  2151191071051219817   
3  2019-10-01 00:00:07+00:00       cart     5723490  1487580005134238553   
4  2019-10-01 00:00:15+00:00       cart     5881449  1487580013522845895   

    brand  price    user_id                          user_session  \
0  runail   2.62  463240011  26dd6e6e-4dac-4778-8d2c-92e149dab885   
1  runail   2.62  463240011  26dd6e6e-4dac-4778-8d2c-92e149dab885   
2  lovely  13.48  429681830  49e8d843-adf3-428b-a2c3-fe8bc6a307c9   
3  runail   2.62  463240011  26dd6e6e-4dac-4778-8d2c-92e149dab885   
4  lovely   0.56  429681830  49e8d843-adf3-428b-a2c3-fe8bc6a307c9   

           event_time_moscow  event_month  \
0  2019-10-01 03:00:00+03:00         10.0   
1  2019-10-01 03:00:03+03:00         1

In [2]:
import pandas as pd

# CSV 또는 Parquet 파일을 불러온 뒤 df에 저장했다고 가정
# 예: df = pd.read_csv('your_file.csv') 또는 pd.read_parquet('your_file.parquet')

#  1. price가 0인 행 제거
df = df[df['price'] > 0]

#  2. brand 컬럼에서 결측치 제거
df = df.dropna(subset=['brand'])

# 결과 확인 (선택사항)
print(df.shape)
print(df['price'].min())
print(df['brand'].isnull().sum())

(11932733, 11)
0.06
0


In [3]:
import pandas as pd

# 1. 구매 이벤트만 필터링
purchase_df = df[df['event_type'] == 'purchase']

# 2. 사용자-상품별 구매 횟수 집계
user_product_counts = purchase_df.groupby(['user_key', 'product_id']).size().reset_index(name='purchase_count')

# 3. 사용자별 재구매 여부 판단 (동일 상품 2회 이상 구매한 경우)
user_product_counts['is_repeat'] = user_product_counts['purchase_count'] >= 2

# 4. 사용자별 재구매 상품 수 집계
user_repeat_summary = user_product_counts.groupby('user_key')['is_repeat'].sum().reset_index()
user_repeat_summary.rename(columns={'is_repeat': 'repeat_product_count'}, inplace=True)

# 5. 사용자별 전체 구매 상품 수 집계
user_total_product = purchase_df.groupby('user_key')['product_id'].nunique().reset_index()
user_total_product.rename(columns={'product_id': 'total_product_count'}, inplace=True)

# 6. 병합 후 재구매율 계산
user_repeat_rate = pd.merge(user_total_product, user_repeat_summary, on='user_key', how='left')
user_repeat_rate['repeat_product_count'] = user_repeat_rate['repeat_product_count'].fillna(0)
user_repeat_rate['repeat_rate'] = (user_repeat_rate['repeat_product_count'] / user_repeat_rate['total_product_count']) * 100
user_repeat_rate['repeat_rate'] = user_repeat_rate['repeat_rate'].round(2)

# 7. 결과 출력
print("✅ 사용자별 재구매율 (상위 10명 예시):")
print(user_repeat_rate.sort_values('repeat_rate', ascending=False).head(10))

# 8. CSV 저장
user_repeat_rate.to_csv("user_repeat_purchase_rate.csv", index=False)
print("'user_repeat_purchase_rate.csv' 저장 완료!")


✅ 사용자별 재구매율 (상위 10명 예시):
                                            user_key  total_product_count  \
35218  477917815a5108d32-1e27-442a-9421-d972009d87ee                    4   
29348  462339675c2a67092-e0f2-49e1-806f-00b313c3ec51                   20   
38644  488102643830df9ca-0a1a-4323-bd01-7fcc717f154e                    1   
75427  55724863659f54d11-3edd-433d-b0a7-6e0135a4eaef                    1   
57333  5351713585b64d5ad-faf6-424a-b732-24533d1533e3                    2   
94387  5704489970e56b6ca-1f87-48f7-b6b9-0fd4371fd6e5                    3   
78871  559439821d63d79be-4e1b-4455-bddf-84a21569891c                    1   
26754  45206465107d21a96-d02e-4a69-8329-4a71a0663bf2                   12   
18413  417009336806f5f21-5ece-4ac7-be80-87823886dc9e                    4   
12160  379812188ec34601f-f09e-699c-ba67-340ffdbe23ee                    4   

       repeat_product_count  repeat_rate  
35218                     4        100.0  
29348                    20        100.0 

In [4]:
import pandas as pd

# 1. 구매 이벤트만 필터링
purchase_df = df[df['event_type'] == 'purchase']

# 2. 사용자별 서로 다른 상품 구매 개수 계산
user_product_counts = purchase_df.groupby('user_key')['product_id'].nunique().reset_index()
user_product_counts.rename(columns={'product_id': 'unique_product_count'}, inplace=True)

# 3. 재구매 고객 (2개 이상 상품 구매한 경우)
user_product_counts['is_repeat_customer'] = user_product_counts['unique_product_count'] >= 2

# 4. 재구매 고객 수 & 전체 구매 고객 수
total_buyers = len(user_product_counts)
repeat_buyers = user_product_counts['is_repeat_customer'].sum()
repeat_rate = repeat_buyers / total_buyers * 100 if total_buyers > 0 else 0

# 5. 요약 결과 DataFrame
summary_df = pd.DataFrame({
    'total_buyers': [total_buyers],
    'repeat_buyers': [repeat_buyers],
    'repeat_rate(%)': [round(repeat_rate, 2)]
})

# 6. 저장
summary_df.to_csv('repeat_customer_rate.csv', index=False)
user_product_counts.to_csv('user_level_repeat_flag.csv', index=False)

# 7. 출력
print("✅ 고객 기준 재구매율 요약:")
print(summary_df)

print("\n📄 고객별 재구매 여부 (상위 10개):")
print(user_product_counts.head(10))


✅ 고객 기준 재구매율 요약:
   total_buyers  repeat_buyers  repeat_rate(%)
0        137908         112198           81.36

📄 고객별 재구매 여부 (상위 10개):
                                        user_key  unique_product_count  \
0  100557945a8fa67f1-1321-e647-3038-009b16406a19                     3   
1  100557945b1845d34-1af1-45f7-8948-2baa55e934cc                     2   
2  1007877819a95e186-1c1a-4ea0-8559-600df69368e3                     3   
3   10079204f873d16b-4efa-4e92-81be-92948aef2ff5                     2   
4  1010254164c6468ec-42fe-489c-bc2c-9fbc31e5eb59                     7   
5  10128176312108fb2-3939-40d4-bc61-3e656a08b115                     1   
6  10157148669be3bbc-c7e0-47f1-8749-94640e2f98ec                     5   
7  101779631be7712e7-acfa-4c8b-882b-4d5c40bcc32c                     7   
8  1027718682107e78d-e00d-4608-bf47-c7fb3a5cba81                     2   
9  102771868609fe381-dcbf-4bb3-bec2-5325c3296be9                     2   

   is_repeat_customer  
0                True  
1 

In [5]:
import pandas as pd

# 1. 구매 이벤트 필터링
purchase_df = df[df['event_type'] == 'purchase'].copy()

# 2. 구매 일자 컬럼 생성
purchase_df['purchase_date'] = pd.to_datetime(purchase_df['event_time']).dt.date

# 3. 고객별 구매한 서로 다른 날짜 수 계산
user_purchase_days = purchase_df.groupby('user_key')['purchase_date'].nunique().reset_index()
user_purchase_days.rename(columns={'purchase_date': 'unique_purchase_days'}, inplace=True)

# 4. 재구매 고객 여부 판별 (2일 이상 구매 → 재구매로 간주)
user_purchase_days['is_repeat_customer'] = user_purchase_days['unique_purchase_days'] >= 2

# 5. 재구매율 계산
total_buyers = len(user_purchase_days)
repeat_buyers = user_purchase_days['is_repeat_customer'].sum()
repeat_rate = repeat_buyers / total_buyers * 100 if total_buyers > 0 else 0

# 6. 요약 결과
summary_df = pd.DataFrame({
    'total_buyers': [total_buyers],
    'repeat_buyers': [repeat_buyers],
    'repeat_rate(%)': [round(repeat_rate, 2)]
})

# 7. 저장
summary_df.to_csv('repeat_customer_rate_by_day.csv', index=False)
user_purchase_days.to_csv('user_level_repeat_flag_by_day.csv', index=False)

# 8. 출력
print("✅ 서로 다른 날짜 기준 재구매율 요약:")
print(summary_df)

print("\n📄 고객별 재구매 여부 (상위 10개):")
print(user_purchase_days.head(10))


✅ 서로 다른 날짜 기준 재구매율 요약:
   total_buyers  repeat_buyers  repeat_rate(%)
0        137908            644            0.47

📄 고객별 재구매 여부 (상위 10개):
                                        user_key  unique_purchase_days  \
0  100557945a8fa67f1-1321-e647-3038-009b16406a19                     1   
1  100557945b1845d34-1af1-45f7-8948-2baa55e934cc                     1   
2  1007877819a95e186-1c1a-4ea0-8559-600df69368e3                     1   
3   10079204f873d16b-4efa-4e92-81be-92948aef2ff5                     1   
4  1010254164c6468ec-42fe-489c-bc2c-9fbc31e5eb59                     1   
5  10128176312108fb2-3939-40d4-bc61-3e656a08b115                     1   
6  10157148669be3bbc-c7e0-47f1-8749-94640e2f98ec                     1   
7  101779631be7712e7-acfa-4c8b-882b-4d5c40bcc32c                     1   
8  1027718682107e78d-e00d-4608-bf47-c7fb3a5cba81                     1   
9  102771868609fe381-dcbf-4bb3-bec2-5325c3296be9                     1   

   is_repeat_customer  
0               Fals

In [6]:
import pandas as pd

# 1. 구매 데이터 필터링
purchase_df = df[df['event_type'] == 'purchase'].copy()
purchase_df['purchase_date'] = pd.to_datetime(purchase_df['event_time']).dt.date

# 2. 고객별 구매 날짜 정보
purchase_days = purchase_df.groupby('user_key')['purchase_date'].agg(['min', 'max', 'nunique']).reset_index()
purchase_days.rename(columns={'min': 'first_purchase', 'max': 'last_purchase', 'nunique': 'unique_purchase_days'}, inplace=True)

# 3. 재구매 고객만 필터링
repeat_customers = purchase_days[purchase_days['unique_purchase_days'] >= 2].copy()

# ✅ 4. 재구매 주기 계산 (에러 수정)
repeat_customers['purchase_interval_days'] = (
    (repeat_customers['last_purchase'] - repeat_customers['first_purchase']).apply(lambda x: x.days)
) / (repeat_customers['unique_purchase_days'] - 1)

repeat_customers['purchase_interval_days'] = repeat_customers['purchase_interval_days'].round(2)

# 5. 저장
repeat_customers.to_csv('repeat_customers_purchase_interval.csv', index=False)

# 6. 출력
print("📊 재구매 고객 상위 10명 재구매 주기:")
print(repeat_customers[['user_key', 'unique_purchase_days', 'purchase_interval_days']].head(10))



📊 재구매 고객 상위 10명 재구매 주기:
                                          user_key  unique_purchase_days  \
64   1144013979412a50b-9465-47c5-889b-8340ec14a4e1                     3   
69   1153985283ad50787-d391-4bea-900e-26c9aee1ed95                     8   
133  1319941386772c164-711e-4b0d-bfc8-a30fcca5a937                     2   
136  132692896d2605f8d-1365-497f-90d9-ece705b66476                     5   
301  15122987263231af6-b308-4430-b661-aac5621956ee                     2   
401  160001681252f82bc-7741-43f5-a2ef-173b6c385b37                     2   
442  16428347289c22c8b-bd99-4fb0-93f9-ce7bc0448c50                     2   
462  1699381791a78d97e-4d77-4cac-a986-81380e8338b4                     6   
491  17297448515c8a6d8-cad6-45c3-aa38-d0b39b8a0c9d                     2   
595  18341265923f45d6b-36f1-434a-beb3-ac94f6d805e4                     2   

     purchase_interval_days  
64                    33.50  
69                    13.86  
133                   35.00  
136                

In [7]:
import pandas as pd

# 1. 구매 이벤트만 필터링
purchase_df = df[df['event_type'] == 'purchase'].copy()

# 2. 사용자-상품-월별 구매 횟수 집계
user_product_counts = purchase_df.groupby(['event_month', 'user_key', 'product_id']).size().reset_index(name='purchase_count')

# 3. 재구매 여부 판단 (같은 월 내 동일 상품을 2회 이상 구매한 경우)
user_product_counts['is_repeat'] = user_product_counts['purchase_count'] >= 2

# 4. 사용자-월별 재구매 상품 수
user_repeat_summary = user_product_counts.groupby(['event_month', 'user_key'])['is_repeat'].sum().reset_index()
user_repeat_summary.rename(columns={'is_repeat': 'repeat_product_count'}, inplace=True)

# 5. 사용자-월별 전체 구매 상품 수
user_total_product = purchase_df.groupby(['event_month', 'user_key'])['product_id'].nunique().reset_index()
user_total_product.rename(columns={'product_id': 'total_product_count'}, inplace=True)

# 6. 병합 후 재구매율 계산
user_repeat_rate = pd.merge(user_total_product, user_repeat_summary, on=['event_month', 'user_key'], how='left')
user_repeat_rate['repeat_product_count'] = user_repeat_rate['repeat_product_count'].fillna(0)
user_repeat_rate['repeat_rate'] = (user_repeat_rate['repeat_product_count'] / user_repeat_rate['total_product_count']) * 100
user_repeat_rate['repeat_rate'] = user_repeat_rate['repeat_rate'].round(2)

# 7. 저장
user_repeat_rate.to_csv("monthly_user_repeat_purchase_rate.csv", index=False)
print("✅ 'monthly_user_repeat_purchase_rate.csv' 저장 완료!")

# 8. 미리보기: 각 월 상위 3명 출력
print("🎯 월별 사용자 재구매율 미리보기:")
print(user_repeat_rate.groupby('event_month').head(3))


✅ 'monthly_user_repeat_purchase_rate.csv' 저장 완료!
🎯 월별 사용자 재구매율 미리보기:
        event_month                                       user_key  \
0               1.0  10157148669be3bbc-c7e0-47f1-8749-94640e2f98ec   
1               1.0  101779631be7712e7-acfa-4c8b-882b-4d5c40bcc32c   
2               1.0   10280338c9cad2fe-9213-42c0-a68a-c329b62f394a   
28878           2.0  100557945a8fa67f1-1321-e647-3038-009b16406a19   
28879           2.0  1027718682107e78d-e00d-4608-bf47-c7fb3a5cba81   
28880           2.0  102771868609fe381-dcbf-4bb3-bec2-5325c3296be9   
54835          10.0  10458417587a7b93b-d9a1-414f-90f5-3d6da9139528   
54836          10.0  1050754406e6853cf-491b-412a-8083-2725a2f42f5b   
54837          10.0  10511820319a9c2e1-ed34-4680-8bc9-430c32df7073   
80841          11.0  100557945b1845d34-1af1-45f7-8948-2baa55e934cc   
80842          11.0  1007877819a95e186-1c1a-4ea0-8559-600df69368e3   
80843          11.0   10079204f873d16b-4efa-4e92-81be-92948aef2ff5   
113452         12.0  

In [8]:
import pandas as pd

# 1. 구매 이벤트 필터링
purchase_df = df[df['event_type'] == 'purchase'].copy()

# 2. 날짜 컬럼 처리
purchase_df['purchase_date'] = pd.to_datetime(purchase_df['event_time']).dt.date

# 3. 월별 고객별 서로 다른 구매 날짜 수 계산
monthly_purchase_days = purchase_df.groupby(['event_month', 'user_key'])['purchase_date'].nunique().reset_index()
monthly_purchase_days.rename(columns={'purchase_date': 'unique_purchase_days'}, inplace=True)

# 4. 재구매 고객 여부 (2일 이상 구매)
monthly_purchase_days['is_repeat_customer'] = monthly_purchase_days['unique_purchase_days'] >= 2

# 5. 월별 재구매율 요약 집계
summary = monthly_purchase_days.groupby('event_month').agg(
    total_buyers=('user_key', 'nunique'),
    repeat_buyers=('is_repeat_customer', 'sum')
).reset_index()

summary['repeat_rate(%)'] = (summary['repeat_buyers'] / summary['total_buyers'] * 100).round(2)

# 6. 저장
summary.to_csv('monthly_repeat_customer_rate_by_day.csv', index=False)
monthly_purchase_days.to_csv('monthly_user_level_repeat_flag_by_day.csv', index=False)

# 7. 출력
print("✅ 월별 서로 다른 날짜 기준 재구매율 요약:")
print(summary)

print("\n📄 월별 고객별 재구매 여부 (상위 10개):")
print(monthly_purchase_days.head(10))


✅ 월별 서로 다른 날짜 기준 재구매율 요약:
   event_month  total_buyers  repeat_buyers  repeat_rate(%)
0          1.0         28878             64            0.22
1          2.0         25957             57            0.22
2         10.0         26006             97            0.37
3         11.0         32611            103            0.32
4         12.0         25298             51            0.20

📄 월별 고객별 재구매 여부 (상위 10개):
   event_month                                       user_key  \
0          1.0  10157148669be3bbc-c7e0-47f1-8749-94640e2f98ec   
1          1.0  101779631be7712e7-acfa-4c8b-882b-4d5c40bcc32c   
2          1.0   10280338c9cad2fe-9213-42c0-a68a-c329b62f394a   
3          1.0  103156342c87ad594-4ea6-21f2-3a31-dfeaec25f0ea   
4          1.0  103274988556059be-280e-4cc3-96c3-9004102d856e   
5          1.0  10480826873ecfb92-f37f-465a-a200-cb144adbffd1   
6          1.0  105118203a46ca9a9-3cd2-4d86-8b14-a16e9f511bcd   
7          1.0  10569763038553eef-43b4-4b6d-8768-360b0a9ff644   
8 